# NFL Data Scraping Project
This is a notebook that will describe the web scraping process for NFL team statistics

To start we need to figure out what information and where to find that information. For this project I want to analyze how regular season statistics for a team impact their performance during the regular season and in the playoffs.
- To find the regular season statistics I will be using the information provided by the NFL here: **https://www.nfl.com/stats/team-stats/**
- To find the win and loss information for teams I used information from this link: **https://www.teamrankings.com/nfl/trends/win_trends/**

Let's start by gathering the win loss data

### Packages
For this project I will be using selenium and using beautiful soup to conduct web scraping.


In [9]:
from bs4 import BeautifulSoup
from requests_html import HTMLSession
from urllib.parse import urljoin
import requests as r

url = "https://www.teamrankings.com/nfl/trends/win_trends/"
response = r.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
specific_div = soup.find('table') # finding the table with the record of each team
rows = specific_div.find_all('tr') # get the rows of data
# Now that we have the data as a list of rows, we can parse the data to construct a data frame

[<tr>
<th class="text-left sort-asc-first">Team</th>
<th class="text-right nowrap">Win-Loss Record</th>
<th class="text-right nowrap sort-first">Win %</th>
<th class="text-right nowrap">MOV</th>
<th class="text-right nowrap">ATS +/-</th>
</tr>, <tr>
<td class="nowrap"><a href="https://www.teamrankings.com/nfl/team/philadelphia-eagles">Philadelphia</a></td>
<td class="text-right" data-sort="8">8-1-0</td>
<td class="text-right green" data-sort="0.8889">88.9%</td>
<td class="text-right" data-sort="6.333333333333333">6.3</td>
<td class="text-right" data-sort="1.1666666666666667">+1.2</td>
</tr>, <tr>
<td class="nowrap"><a href="https://www.teamrankings.com/nfl/team/kansas-city-chiefs">Kansas City</a></td>
<td class="text-right" data-sort="7">7-2-0</td>
<td class="text-right green" data-sort="0.7778">77.8%</td>
<td class="text-right" data-sort="7.222222222222222">7.2</td>
<td class="text-right" data-sort="1.1111111111111112">+1.1</td>
</tr>, <tr>
<td class="nowrap"><a href="https://www.team